# Importando módulos

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Helper Functions

In [2]:
def check_duplicates(df, subset=None):
    """
    Checa se existem duplicações em colunas específicas de um DataFrame.
        - df (Pandas DataFrame): DataFrame em que a função deve checar se há duplicações.
        - subset (list of strings): Colunas em que a função deve checar se há duplicações. Se nenhum valor for passado,
                                    a função vai checar se há linhas completamente duplicadas.
    """
    if (type(subset) not in (list,str)) & (subset != None): raise ValueError('subset deve ser do tipo LIST')
    if type(subset) == str: subset=[subset]
        
    contains_duplicated = False
    
    if subset == None:
        n_duplicated = df.duplicated().sum()
        if n_duplicated != 0:
            contains_duplicated = True
            duplicated_df = df[df.duplicated()]
            print("Há duplicações na(s) linha(s) ")
            for index in duplicated_df.index:
                print(index)
    else:
        for column in subset:
            if column not in df.columns: raise ValueError('Não existe coluna {} no DataFrame indicado'.format(column))
            n_duplicated = df.duplicated(subset=column).sum()
            if n_duplicated != 0:
                contains_duplicated = True
                duplicated_values = df[df.duplicated(subset=column)][column].unique()
                for value in duplicated_values:
                    print("Valor {} duplicado na coluna {}".format(value,column))
    
    if contains_duplicated == False:
        print("Não há duplicações.")

# Carregando DataFrames

In [3]:
excel_file = pd.read_excel("Base de dados Case.xlsx", sheet_name=None)
populacao_estado = excel_file['Populacao_Estado']
de_para_uf = excel_file['De_para_UF']
pib_municipio = excel_file['PIB_municipio']
uf_regiao = excel_file['UF_Regiao']

# Tratamento dos dados

#### Populacao_Estado

In [4]:
populacao_estado

,Granularidade,fx_idade,Populacao
0,Brasil,Total,190755799
1,Brasil,0 a 4 anos,13806733
2,Brasil,5 a 9 anos,14967767
3,Brasil,10 a 14 anos,17167135
4,Brasil,15 a 19 anos,16986788
...,...,...,...
611,Distrito Federal,80 a 84 anos,12252
612,Distrito Federal,85 a 89 anos,5946
613,Distrito Federal,90 a 94 anos,2425
614,Distrito Federal,95 a 99 anos,530


In [5]:
# Checando linhas duplicadas
check_duplicates(populacao_estado)

Não há duplicações.


In [6]:
# Checando valores nulos (0: não há valores nulos | maior que 0: há valores nulos)
populacao_estado.isna().sum()

Granularidade    0
fx_idade         0
Populacao        0
dtype: int64

In [7]:
# criando DataFrame total_pop, que vai conter somente o total da população de cada estado e do país como um todo.
total_pop = populacao_estado[populacao_estado['fx_idade'] == 'Total'].reset_index(drop=True)

In [8]:
total_pop

,Granularidade,fx_idade,Populacao
0,Brasil,Total,190755799
1,Rondonia,Total,1562409
2,Acre,Total,733559
3,Amazonas,Total,3483985
4,Roraima,Total,450479
5,Para,Total,7581051
6,Amapa,Total,669526
7,Tocantins,Total,1383445
8,Maranhao,Total,6574789
9,Piaui,Total,3118360


In [9]:
# Criando o DataFrame uf_pop, que vai conter as populações totais somente dos estados, excluindo a soma do país inteiro.
uf_pop = total_pop[1:].reset_index(drop=True)

In [10]:
# Checando se a população de todos os estados somados é igual à população do país (valor esperado: True)
uf_pop['Populacao'].sum() == total_pop[total_pop['Granularidade'] == 'Brasil']['Populacao'][0]

True

------

#### PIB_municipio

In [11]:
pib_municipio

,Cod_Identificacao,Municipio,PIB
0,1100015,Alta Floresta D'Oeste (RO),262077
1,1100023,Ariquemes (RO),1364694
2,1100031,Cabixi (RO),69611
3,1100049,Cacoal (RO),1186494
4,1100056,Cerejeiras (RO),222021
...,...,...,...
5565,5222005,Vianópolis (GO),178699
5566,5222054,Vicentinópolis (GO),133899
5567,5222203,Vila Boa (GO),71676
5568,5222302,Vila Propício (GO),92063


In [12]:
# Adicionando coluna UF
pib_municipio['UF'] = pib_municipio['Municipio'].str[-3:-1]
pib_municipio

,Cod_Identificacao,Municipio,PIB,UF
0,1100015,Alta Floresta D'Oeste (RO),262077,RO
1,1100023,Ariquemes (RO),1364694,RO
2,1100031,Cabixi (RO),69611,RO
3,1100049,Cacoal (RO),1186494,RO
4,1100056,Cerejeiras (RO),222021,RO
...,...,...,...,...
5565,5222005,Vianópolis (GO),178699,GO
5566,5222054,Vicentinópolis (GO),133899,GO
5567,5222203,Vila Boa (GO),71676,GO
5568,5222302,Vila Propício (GO),92063,GO


In [13]:
# Checando linhas duplicadas
check_duplicates(pib_municipio)

Não há duplicações.


In [14]:
# Checando valores duplicados por colunas
check_duplicates(pib_municipio, subset=['Cod_Identificacao','Municipio'])

Não há duplicações.


In [15]:
# Checando valores nulos (0: não há valores nulos | maior que 0: há valores nulos)
pib_municipio.isna().sum()

Cod_Identificacao    0
Municipio            0
PIB                  0
UF                   0
dtype: int64

In [16]:
# Checando se a coluna UF possui 27 valores únicos
len(pib_municipio['UF'].unique()) == 27

True

In [17]:
# Checando o tipo da coluna PIB
pib_municipio['PIB'].dtype

dtype('O')

In [18]:
# A coluna PIB consta como Object (string). Vamos alterá-la para int, para que assim possamos somar seus valores.

# Checando possíveis valores não númericos
pib_municipio['numeric_check'] = pib_municipio['PIB'].str.isnumeric()
pib_municipio[pib_municipio['numeric_check'] == False]

,Cod_Identificacao,Municipio,PIB,UF,numeric_check
224,1504752,Mojuí dos Campos (PA),...,PA,False
4503,4212650,Pescaria Brava (SC),...,SC,False
4605,4220000,Balneário Rincão (SC),...,SC,False
4923,4314548,Pinto Bandeira (RS),...,RS,False
5160,5006275,Paraíso das Águas (MS),...,MS,False


In [19]:
# Fazendo uma checagem no site do IBGE, descobrimos que os 5 municípios acima, que não possuem dados acerca do PIB,
# foram criados após o Censo de 2010. Assim sendo, removeremos estas 5 observações,
# visto que não foram computadas no cálculo do PIB per capita do referido ano.

pib_municipio = pib_municipio[pib_municipio['numeric_check'] != False].reset_index(drop=True)
pib_municipio.drop(columns='numeric_check', inplace=True)

In [20]:
# Vamos checar a escala em que se encontra o PIB analisando o PIB de São Paulo,
# sabendo que em 2010, o PIB da cidade foi em torno de 450 bilhões de reais
pib_municipio[pib_municipio['Municipio'] == 'São Paulo (SP)']

,Cod_Identificacao,Municipio,PIB,UF
3828,3550308,São Paulo (SP),450491988,SP


In [21]:
# É possível identificar que a coluna PIB está na unidade (R$ x1000)
# Devemos lembrar disto no momento de calcular o Pib per Capita

In [22]:
# Transformando a coluna PIB em numérica (int)
pib_municipio['PIB'] = pib_municipio['PIB'].astype('int64')

In [23]:
# Checando tipo da coluna PIB
pib_municipio['PIB'].dtype

dtype('int64')

In [24]:
# Agrupando estados por PIB
total_pib = pib_municipio.groupby('UF')['PIB'].agg('sum')

In [25]:
total_pib

UF
AC       8342356
AL      27133035
AM      60877125
AP       8237794
BA     154419544
CE      79336306
DF     144174102
ES      85310281
GO     106770107
MA      46309630
MG     351123420
MS      47270652
MT      56600957
PA      82684518
PB      33522490
PE      97189752
PI      22269147
PR     225205247
RJ     449858101
RN      36184501
RO      23907884
RR       6639150
RS     241249165
SC     153726009
SE      26404894
SP    1294695991
TO      16404814
Name: PIB, dtype: int64

----------

#### De_para_UF

In [26]:
de_para_uf

,Granularidade,UF,Cod_Identificacao
0,Acre,AC,12
1,Alagoas,AL,27
2,Amapa,AP,16
3,Amazonas,AM,13
4,Bahia,BA,29
5,Ceara,CE,23
6,Distrito Federal,DF,53
7,Espirito Santo,ES,32
8,Goias,GO,52
9,Maranhao,MA,21


In [27]:
# Checando linhas duplicadas
check_duplicates(de_para_uf)

Há duplicações na(s) linha(s) 
27


In [28]:
# Visualizando linha duplicada
de_para_uf[de_para_uf.duplicated()]

,Granularidade,UF,Cod_Identificacao
27,Distrito Federal,DF,53


In [29]:
# Removendo linha duplicada
de_para_uf.drop_duplicates(inplace=True)

In [30]:
# Checando novamente por linhas duplicadas
check_duplicates(de_para_uf)

Não há duplicações.


In [31]:
# Já removemos a única linha completamente duplicada do df de_para_uf
# Agora, devemos checar se há valores duplicados nas colunas que não fazem sentido ter repetições

# Checando valores duplicadas por colunas
check_duplicates(de_para_uf, subset=['Granularidade','UF','Cod_Identificacao'])

Valor MG duplicado na coluna UF


In [32]:
# Visualizando dados coletados como MG na coluna UF
de_para_uf[de_para_uf['UF'] == 'MG']

,Granularidade,UF,Cod_Identificacao
10,Mato Grosso,MG,51
12,Minas Gerais,MG,31


In [33]:
# Mato Grosso foi coletado como MG, quando o UF correto é MT

# Corrigindo UF de Mato Grosso para MT
de_para_uf.loc[10,'UF'] = 'MT'

In [34]:
# Checando novamente por valores duplicados nas coluans
check_duplicates(de_para_uf, subset=['Granularidade','UF','Cod_Identificacao'])

Não há duplicações.


In [35]:
# Checando novamente o DF
de_para_uf

,Granularidade,UF,Cod_Identificacao
0,Acre,AC,12
1,Alagoas,AL,27
2,Amapa,AP,16
3,Amazonas,AM,13
4,Bahia,BA,29
5,Ceara,CE,23
6,Distrito Federal,DF,53
7,Espirito Santo,ES,32
8,Goias,GO,52
9,Maranhao,MA,21


#### UF_Regiao

In [36]:
uf_regiao

,Estado,Regiao
0,Rondonia,N
1,Acre,N
2,Amazonas,N
3,Roraima,N
4,Para,N
5,Amapa,N
6,Tocantins,N
7,Maranhao,NE
8,Piaui,NE
9,Ceara,NE


In [37]:
# Checando linhas duplicadas
check_duplicates(uf_regiao)

Não há duplicações.


In [38]:
# Checando valores duplicadas por colunas
check_duplicates(uf_regiao, subset=['Estado'])

Não há duplicações.


# Juntando DataFrames

In [39]:
# Joinning uf_pop, de_para_uf, total_pib e uf_regiao
joined_df = uf_pop\
                .merge(de_para_uf, on='Granularidade')\
                .merge(total_pib, on='UF')\
                .merge(uf_regiao, left_on='Granularidade', right_on='Estado')
joined_df

,Granularidade,fx_idade,Populacao,UF,Cod_Identificacao,PIB,Estado,Regiao
0,Rondonia,Total,1562409,RO,11,23907884,Rondonia,N
1,Acre,Total,733559,AC,12,8342356,Acre,N
2,Amazonas,Total,3483985,AM,13,60877125,Amazonas,N
3,Roraima,Total,450479,RR,14,6639150,Roraima,N
4,Para,Total,7581051,PA,15,82684518,Para,N
5,Amapa,Total,669526,AP,16,8237794,Amapa,N
6,Tocantins,Total,1383445,TO,17,16404814,Tocantins,N
7,Maranhao,Total,6574789,MA,21,46309630,Maranhao,NE
8,Piaui,Total,3118360,PI,22,22269147,Piaui,NE
9,Ceara,Total,8452381,CE,23,79336306,Ceara,NE


In [40]:
# Checando linhas duplicadas
check_duplicates(joined_df)

Não há duplicações.


In [41]:
# Checando valores duplicadas por colunas
check_duplicates(joined_df, subset=['Granularidade','Populacao','UF','Cod_Identificacao','PIB','Estado'])

Não há duplicações.


In [42]:
# Separando colunas que nos interessam
joined_df = joined_df[['Estado','Regiao', 'Populacao','PIB']]
joined_df

,Estado,Regiao,Populacao,PIB
0,Rondonia,N,1562409,23907884
1,Acre,N,733559,8342356
2,Amazonas,N,3483985,60877125
3,Roraima,N,450479,6639150
4,Para,N,7581051,82684518
5,Amapa,N,669526,8237794
6,Tocantins,N,1383445,16404814
7,Maranhao,NE,6574789,46309630
8,Piaui,NE,3118360,22269147
9,Ceara,NE,8452381,79336306


In [43]:
# Agrupando por Região e fazendo o somatório das colunas Populacao e PIB
grouped_df = joined_df.groupby('Regiao')[['Populacao','PIB']].agg('sum')
grouped_df.reset_index(inplace=True)
grouped_df

,Regiao,Populacao,PIB
0,CO,14058094,354815818
1,N,15864454,207093641
2,NE,53081950,522769299
3,S,27386891,620180421
4,SE,80364410,2180987793


# Adicionando linha Total

In [44]:
total_population = populacao_estado.iloc[0,-1]
total_pib = pib_municipio['PIB'].sum()

print(f'População total: {total_population}\nPIB total: {total_pib}')

População total: 190755799
PIB total: 3885846972


In [45]:
total_row = pd.Series(['Total',total_population,total_pib], index=grouped_df.columns)
grouped_df = grouped_df.append(total_row, ignore_index=True)

# Adicionando coluna PIB_per_capita

In [46]:
grouped_df['PIB_per_capita'] = grouped_df['PIB'] / grouped_df['Populacao']
grouped_df

,Regiao,Populacao,PIB,PIB_per_capita
0,CO,14058094,354815818,25.239255
1,N,15864454,207093641,13.053941
2,NE,53081950,522769299,9.848344
3,S,27386891,620180421,22.645156
4,SE,80364410,2180987793,27.138727
5,Total,190755799,3885846972,20.370793


In [47]:
# Como sabemos que o PIB estava na escala de (R$ x1000), devemos multiplicar o valor o PIB per capita por 1000 também
grouped_df['PIB_per_capita'] = round(grouped_df['PIB_per_capita']*1000,2)
grouped_df

,Regiao,Populacao,PIB,PIB_per_capita
0,CO,14058094,354815818,25239.25
1,N,15864454,207093641,13053.94
2,NE,53081950,522769299,9848.34
3,S,27386891,620180421,22645.16
4,SE,80364410,2180987793,27138.73
5,Total,190755799,3885846972,20370.79


# Finalizando case

In [48]:
grouped_df = grouped_df[['Regiao','PIB_per_capita']]
grouped_df

,Regiao,PIB_per_capita
0,CO,25239.25
1,N,13053.94
2,NE,9848.34
3,S,22645.16
4,SE,27138.73
5,Total,20370.79


In [49]:
# Substituindo siglas por nomes por extenso
grouped_df['Regiao'] = grouped_df['Regiao'].str\
                                                .replace('CO','Centro-Oeste')\
                                                .replace('N','Norte')\
                                                .replace('NE','Nordeste')\
                                                .replace('S','Sul')\
                                                .replace('SE','Sudeste')\

grouped_df

,Regiao,PIB_per_capita
0,Centro-Oeste,25239.25
1,Norte,13053.94
2,Nordeste,9848.34
3,Sul,22645.16
4,Sudeste,27138.73
5,Total,20370.79


In [50]:
# Organizando a ordem das observações conforme o solicitado
final_df = grouped_df.reindex([1,2,3,4,0,5]).reset_index(drop=True)
final_df

,Regiao,PIB_per_capita
0,Norte,13053.94
1,Nordeste,9848.34
2,Sul,22645.16
3,Sudeste,27138.73
4,Centro-Oeste,25239.25
5,Total,20370.79


---------

Muitíssimo obrigado pela conversa de hoje e pela oportunidade de mostrar um pouco do meu trabalho. Aguardo o retorno!<br>
Bom final de semana!

Matheus Figueiredo<br>
(88) 99773-8175<br>
contact@mathfigueiredo.com